The goal of this notebook is to come up with an algorithm to re-format original and target rhythms such that real-time time-stretching (or even real-time stuttering) is possible.

In [17]:
import librosa
import numpy as np

import pardir; pardir.pardir() # Allow imports from parent directory
import bjorklund
import fibonaccistretch

In [2]:
librosa.effects.time_stretch??
librosa.core.phase_vocoder??
fibonaccistretch.euclidean_stretch??

In [8]:
# Generate rhythms based on parameters
def generate_original_and_target_rhythms(num_pulses, original_length, target_length):
    original_rhythm = bjorklund.bjorklund(pulses=num_pulses, steps=original_length)
    target_rhythm = bjorklund.bjorklund(pulses=len(original_rhythm), steps=target_length)
    return (original_rhythm, target_rhythm)

original_rhythm, target_rhythm = generate_original_and_target_rhythms(3, 8, 13)

"Original rhythm: {}    Target rhythm: {}".format(original_rhythm, target_rhythm)

'Original rhythm: [1, 0, 0, 1, 0, 0, 1, 0]    Target rhythm: [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]'

In [11]:
lcm = (8*13) / fibonaccistretch.euclid(8, 13)
lcm

104

In [12]:
8*13

104

Use LCM to "equalize" rhythms so that they're of equal length.

e.g.

`a = [1,0,0,1]`



`b = [1,1,0]`

become

`equalized_a = [1,-,-,0,-,-,0,-,-,1,-,-]`



`equalized_b = [1,-,-,-,1,-,-,-,0,-,-,-]`

In [29]:
# "Equalize" (i.e. scale rhythms so they're of equal length)
def equalize_rhythm_subdivisions(original_rhythm, target_rhythm, delimiter="-"):
    original_length = len(original_rhythm)
    target_length = len(target_rhythm)
    lcm = (original_length*target_length) / fibonaccistretch.euclid(original_length, target_length)
    original_scale_factor = (lcm / original_length) - 1
    target_scale_factor = (lcm / target_length) - 1
    
    print("lcm={}, original_scale_factor={}, target_scale_factor={}").format(lcm, original_scale_factor, target_scale_factor)
    
    delimiter = str(delimiter)
    original_rhythm = list((delimiter*original_scale_factor).join([str(x) for x in original_rhythm]))
    target_rhythm = list((delimiter*target_scale_factor).join([str(x) for x in target_rhythm]))
    
    original_rhythm.extend(list(delimiter*original_scale_factor))
    target_rhythm.extend(list(delimiter*target_scale_factor))

    
    return (original_rhythm, target_rhythm)

# print(scale_rhythm_subdivisions(original_rhythm, target_rhythm))
original_rhythm, target_rhythm = generate_original_and_target_rhythms(3, 8, 13)
print("Original rhythm: {}    Target rhythm: {}".format(original_rhythm, target_rhythm))
equalized_original_rhythm, equalized_target_rhythm = equalize_rhythm_subdivisions(original_rhythm, target_rhythm)
(len(equalized_original_rhythm), len(equalized_target_rhythm))

Original rhythm: [1, 0, 0, 1, 0, 0, 1, 0]    Target rhythm: [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
lcm=104, original_scale_factor=12, target_scale_factor=7


(104, 104)

Get pulse indices so we can see how the equalized original and target relate. In particular, our goal is to create a relationship such that the original pulse indices always come first (so that they're bufferable in real-time)

In [26]:
def get_pulse_indices_for_rhythm(rhythm, pulse_symbols=[1]):
    pulse_symbols = [str(s) for s in pulse_symbols]
    rhythm = [str(x) for x in rhythm]
    pulse_indices = [i for i,symbol in enumerate(rhythm) if symbol in pulse_symbols]
    return pulse_indices

equalized_original_pulse_indices = get_pulse_indices_for_rhythm(equalized_original_rhythm)
equalized_target_pulse_indices = get_pulse_indices_for_rhythm(equalized_target_rhythm)
(equalized_original_pulse_indices, equalized_target_pulse_indices)

([0, 39, 78], [0, 16, 24, 40, 56, 64, 80, 96])

For original we'll actually use ALL the steps instead of just pulses though. So:

In [28]:
equalized_original_pulse_indices = get_pulse_indices_for_rhythm(equalized_original_rhythm, [1,0])
equalized_target_pulse_indices = get_pulse_indices_for_rhythm(equalized_target_rhythm, [1])
print(equalized_original_pulse_indicesequalized_target_pulse_indices)

[0, 13, 26, 39, 52, 65, 78, 91]
[0, 16, 24, 40, 56, 64, 80, 96]


Now we can check to see if all the original pulse indices come first (this is our goal):

In [35]:
for i in range(len(equalized_original_pulse_indices)):
    opi = equalized_original_pulse_indices[i]
    tpi = equalized_target_pulse_indices[i]
    if (opi > tpi):
        print("Oh no; original pulse at {} comes after target pulse at {} (diff={})".format(opi, tpi, opi-tpi))

Oh no; original pulse at 26 comes after target pulse at 24 (diff=2)
Oh no; original pulse at 65 comes after target pulse at 64 (diff=1)


Oh no... how do we fix this??

- One solution is to just nudge them over, especially since they only differ by 1/104 to 2/104ths of a measure in this case. 
- Another solution would be to use the same data from the original pulse if there's not a new pulse available. Hmmmm
- Or use as much of the original buffer as we can...?

But first let's define a function for end-to-end formatting:

In [70]:
# Format original and target rhythms for real-time manipulation
def format_rhythm(original_rhythm, target_rhythm):
    # Equalize rhythm lengths and get pulse indices
    eor, etr = equalize_rhythm_subdivisions(original_rhythm, target_rhythm)
    eopi, etpi = get_pulse_indices_for_rhythm(eor), get_pulse_indices_for_rhythm(etr)

    # Find all the ones with problematic pulses (note that we're using *pulses* of target but *steps* of original)
    for i in range(min(len(eopi), len(etpi))):
        opi = eopi[i]
        tpi = etpi[i]
        if (opi > tpi):
            print("Oh no; original pulse at {} comes after target pulse at {} (diff={})".format(opi, tpi, opi-tpi))
            
    # TODO: Fix problematic pulses
    #
    
    print("Formatted original: {}".format(rtos(eor)))
    print("Formatted target:   {}".format(rtos(etr)))
    
    return (eor, etr)

# Rhythm to string
def rtos(rhythm):
    return "".join(rhythm)

Alright let's try this out:

In [71]:
# len(original) > len(target)
formatted = format_rhythm([1,0,0,1,0,0,1,0], [1,0,1])

lcm=24, original_scale_factor=2, target_scale_factor=7
Formatted original: 1--0--0--1--0--0--1--0--
Formatted target:   1-------0-------1-------


In [72]:
# len(original) < len(target)
formatted = format_rhythm([1,0,0,1,0,0,1,0], [1,0,0,1,1,0,0,1,1,1,1])

lcm=88, original_scale_factor=10, target_scale_factor=7
Oh no; original pulse at 33 comes after target pulse at 24 (diff=9)
Oh no; original pulse at 66 comes after target pulse at 32 (diff=34)
Formatted original: 1----------0----------0----------1----------0----------0----------1----------0----------
Formatted target:   1-------0-------0-------1-------1-------0-------0-------1-------1-------1-------1-------


In [79]:
# Trying [1,0,1,0] and [1,1] as originals, with the same target
formatted = format_rhythm([1,0,1,0], [1,0,0,1,0,0,1,0,0,0])
print("--------")
formatted = format_rhythm([1,1], [1,0,0,1,0,0,1,0,0,0])

lcm=20, original_scale_factor=4, target_scale_factor=1
Oh no; original pulse at 10 comes after target pulse at 6 (diff=4)
Formatted original: 1----0----1----0----
Formatted target:   1-0-0-1-0-0-1-0-0-0-
--------
lcm=10, original_scale_factor=4, target_scale_factor=0
Oh no; original pulse at 5 comes after target pulse at 3 (diff=2)
Formatted original: 1----1----
Formatted target:   1001001000


To make things a bit clearer maybe we'll try the `abcd` format for `rtos()`

In [100]:
# Rhythm to string
# Method: "str", "alphabet"
def rtos(rhythm, format_method="str", pulse_symbols=["1"]):
    pulse_symbols = [str(s) for s in pulse_symbols]
    
    if format_method == "str":
        return "".join(rhythm)
    elif format_method == "alphabet":
        alphabet = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
        output = []
        ai = 0
        for i,x in enumerate(rhythm):
            if str(x) in pulse_symbols:
                output.append(alphabet[ai%len(alphabet)])
                ai += 1
            else:
                output.append(x)
        return "".join(output)
    else:
        return rhythm
    
# Format original and target rhythms for real-time manipulation
def format_rhythm(original_rhythm, target_rhythm, format_method="str", pulse_symbols=["1"]):
    # Equalize rhythm lengths and get pulse indices
    eor, etr = equalize_rhythm_subdivisions(original_rhythm, target_rhythm)
    eopi, etpi = get_pulse_indices_for_rhythm(eor), get_pulse_indices_for_rhythm(etr)

    # Find all the ones with problematic pulses (note that we're using *pulses* of target but *steps* of original)
    for i in range(min(len(eopi), len(etpi))):
        opi = eopi[i]
        tpi = etpi[i]
        if (opi > tpi):
            print("Oh no; original pulse at {} comes after target pulse at {} (diff={})".format(opi, tpi, opi-tpi))
            
    # TODO: Fix problematic pulses
    #
    
    print("Formatted original: {}".format(rtos(eor, format_method=format_method, pulse_symbols=[1,0])))
    print("Formatted target:   {}".format(rtos(etr, format_method=format_method, pulse_symbols=[1])))
    
    return (eor, etr)

In [101]:
# Trying [1,0,1,0] and [1,1] as originals, with the same target
formatted = format_rhythm([1,0,1,0], [1,0,0,1,0,0,1,0,0,0], format_method="alphabet")
print("--------")
formatted = format_rhythm([1,1], [1,0,0,1,0,0,1,0,0,0], format_method="alphabet")

lcm=20, original_scale_factor=4, target_scale_factor=1
Oh no; original pulse at 10 comes after target pulse at 6 (diff=4)
Formatted original: A----B----C----D----
Formatted target:   A-0-0-B-0-0-C-0-0-0-
--------
lcm=10, original_scale_factor=4, target_scale_factor=0
Oh no; original pulse at 5 comes after target pulse at 3 (diff=2)
Formatted original: A----B----
Formatted target:   A00B00C000


In [102]:
formatted = format_rhythm(original_rhythm, target_rhythm, format_method="alphabet")

lcm=104, original_scale_factor=12, target_scale_factor=7
Oh no; original pulse at 39 comes after target pulse at 16 (diff=23)
Oh no; original pulse at 78 comes after target pulse at 24 (diff=54)
Formatted original: A------------B------------C------------D------------E------------F------------G------------H------------
Formatted target:   A-------0-------B-------C-------0-------D-------0-------E-------F-------0-------G-------0-------H-------


In [18]:
np.indices?

In [14]:
"0"*4
"-".join([str(x) for x in [2,3,4]])

'2-3-4'

In [15]:
print("a"); print("b")

a
b
